In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from math import ceil
import time
import os
import win32com.client

headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.41 YaBrowser/21.2.0.2458 Yowser/2.5 Safari/537.36'
}

#### Этап 1. Собираем ссылки на дела

In [3]:
def get_n_pages_MGS(article, year):
    article = str(article)
    if ', ' in article:
        article = article.replace(', ', '%2C+')
    first_page_url = f'https://mos-gorsud.ru/search?formType=fullForm&courtAlias=&uid=&instance=1&processType=6&letterNumber=&caseNumber=&participant=&codex={article}&judge=&publishingState=&documentType=ab1da71&documentText=&year={year}&caseDateFrom=&caseDateTo=&caseFinalDateFrom=&caseFinalDateTo=&caseLegalForceDateFrom=&caseLegalForceDateTo=&docsDateFrom=&docsDateTo=&documentStatus=2&page=1'
    global headers
    response = requests.get(first_page_url, headers=headers)
    soup = BeautifulSoup(response.text)
    try:
        n_cases = int(soup.find('div', {'class':'resultsearch_text'}).get_text().strip().split('\n')[0].split(': ')[1])
    except:
        n_cases = 0
    n_pages = ceil(n_cases/15)
    return n_pages

In [4]:
def build_url(article, year, page=1):
    article = str(article)
    if ', ' in article:
        article = article.replace(', ', '%2C+')
    url = f'https://mos-gorsud.ru/search?formType=fullForm&courtAlias=&uid=&instance=1&processType=6&letterNumber=&caseNumber=&participant=&codex={article}&judge=&publishingState=&documentType=ab1da71&documentText=&year={year}&caseDateFrom=&caseDateTo=&caseFinalDateFrom=&caseFinalDateTo=&caseLegalForceDateFrom=&caseLegalForceDateTo=&docsDateFrom=&docsDateTo=&documentStatus=2&page={page}'
    return url

In [5]:
def parse_one_page_MGS(url):
    global headers
    response = requests.get(url, headers=headers)
    table = pd.read_html(response.text)[0]
    soup = BeautifulSoup(response.text)
    nobrs = [tag for tag in soup.find('tbody').find_all('nobr') if tag.find('a') is not None]
    urls = ['https://mos-gorsud.ru' + tag.a.get('href').split('?')[0] for tag in nobrs]
    table['URL'] = urls
    return table

In [6]:
articles = ['105', '107', '111, ч.4']
year = 2018
results = pd.DataFrame()

for article in articles:
    n_pages = get_n_pages_MGS(article, year)
    
    for page in range(1, n_pages+1):
        print(f'Статья {article}, год {year}, страница {page}/{n_pages}')
        url = build_url(article, year, page)
        page_results = parse_one_page_MGS(url)
        results = pd.concat([results, page_results])
results = results.reset_index(drop=True)
results.head()

Статья 105, год 2018, страница 1/13
Статья 105, год 2018, страница 2/13
Статья 105, год 2018, страница 3/13
Статья 105, год 2018, страница 4/13
Статья 105, год 2018, страница 5/13
Статья 105, год 2018, страница 6/13
Статья 105, год 2018, страница 7/13
Статья 105, год 2018, страница 8/13
Статья 105, год 2018, страница 9/13
Статья 105, год 2018, страница 10/13
Статья 105, год 2018, страница 11/13
Статья 105, год 2018, страница 12/13
Статья 105, год 2018, страница 13/13
Статья 107, год 2018, страница 1/1
Статья 111, ч.4, год 2018, страница 1/6
Статья 111, ч.4, год 2018, страница 2/6
Статья 111, ч.4, год 2018, страница 3/6
Статья 111, ч.4, год 2018, страница 4/6
Статья 111, ч.4, год 2018, страница 5/6
Статья 111, ч.4, год 2018, страница 6/6


,Номер дела ~ материала,Стороны,Текущее состояние,Судья,Статья,Категория дела,Список дел,URL
0,01-0830/2018,"Подсудимый: Коловертов А.В. (Ст. 105, Ч. 1;)","Вступило в силу, 17.04.2019",Филатов А.Ю.,NaN,NaN,Уголовные дела первой инстанции (Нагатинский р...,https://mos-gorsud.ru/rs/nagatinskij/services/...
1,01-0808/2018,"Подсудимый: Жаров О.В. (Ст. 105, Ч. 1;)","Вступило в силу, 10.01.2019",Кузнецов В.С.,NaN,NaN,Уголовные дела первой инстанции (Люблинский ра...,https://mos-gorsud.ru/rs/lyublinskij/services/...
2,01-0804/2018,"Подсудимый: Яровенко С.А. (Ст. 105, Ч. 1;)","Вступило в силу, 20.02.2019",Крутовская Л.А.,NaN,NaN,Уголовные дела первой инстанции (Нагатинский р...,https://mos-gorsud.ru/rs/nagatinskij/services/...
3,01-0796/2018,"Подсудимый: Васильев Н.Б. (Ст. 105, Ч. 1;)","Вступило в силу, 23.10.2018",Борисенкова Н.В.,NaN,NaN,Уголовные дела первой инстанции (Нагатинский р...,https://mos-gorsud.ru/rs/nagatinskij/services/...
4,01-0731/2018,"Подсудимый: Герасимова Е.А. (Ст. 105, Ч. 1;)","Вступило в силу, 25.02.2019",Филатов А.Ю.,NaN,NaN,Уголовные дела первой инстанции (Нагатинский р...,https://mos-gorsud.ru/rs/nagatinskij/services/...


#### Этап 2. Добавляем ссылки на тексты приговоров

In [7]:
def get_verdict_url(case_url):
    global headers
    case_url += '#tabs-3'
    response = requests.get(case_url, headers=headers)
    soup = BeautifulSoup(response.text)
    tags_with_files = [tag for tag in soup.find_all('tbody')[-1].find_all('tr') if tag.find('a') is not None]
    verdict_file = [tag.a.get('href') for tag in tags_with_files if 'Приговор' in tag.text][0]
    verdict_file = 'https://mos-gorsud.ru' + verdict_file
    return verdict_file

In [9]:
n = len(results)
for i in results.index:
    if i==0 or (i+1)%25==0:
        #print(f'{i+1}/{n}')
    case_url = results.loc[i, 'URL']
    try:
        results.loc[i, 'URL_verdict'] = get_verdict_url(case_url)
    except:
        results.loc[i, 'URL_verdict'] = None
    time.sleep(0.3)

In [8]:
results.to_excel(f'MOSGORSUD_urls_{year}.xlsx')

#### Этап 3. Добавляем тексты приговоров

In [10]:
year = 2018
results = pd.read_excel(f'MOSGORSUD_urls_{year}.xlsx', index_col=0)
results['custom_id'] = results['URL'].apply(lambda x: x.split('/')[-1])

In [11]:
def get_text_by_url_MGS(url, verdict_id, folder='MOSGORSUD_CASES'):
    global headers
    filename = f'{verdict_id}.doc'
    if folder not in os.listdir():
        os.mkdir(folder)
    if filename not in os.listdir(folder):    
        response = requests.get(url, headers=headers)
        with open(folder + "\\" + filename, 'wb') as file:
            file.write(response.content)
    word = win32com.client.Dispatch("Word.Application")
    word.visible = False
    wb = word.Documents.Open(os.getcwd() + "\\" + folder + "\\" + filename)
    doc = word.ActiveDocument
    text = doc.Range().Text
    return text

In [11]:
for i in results.index:
    if i==0 or i%25==0:
        #print(f'{i+1}/{len(results)}')
    verdict_url = results.loc[i, 'URL_verdict']
    verdict_id = results.loc[i, 'custom_id']
    results.loc[i, 'text'] = get_text_by_url_MGS(verdict_url, verdict_id)

In [14]:
results['text'] = results['text'].str.replace('\r', ' ')
results.to_excel(f'MOSGORSUD_texts_{year}.xlsx')